In [1]:
# Suppression des avertissements
import warnings
warnings.filterwarnings('ignore')

# Bibliothèques standards
import numpy as np
import pandas as pd
from datetime import datetime

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Prétraitement des données
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE

# Séparation des données et validation croisée
from sklearn.model_selection import train_test_split, StratifiedKFold

# Modèles de classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Évaluation des modèles
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    f1_score,
    roc_auc_score,
    average_precision_score
)


In [2]:
# Importer le dataset
df1 = pd.read_excel('data/données_chèque_stage.xls')
df2 = pd.read_excel('data/données_LDC_stage.xls')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [3]:
# Vérifier les premières lignes
df1.head()

# Vérifier les informations sur le dataset
df1.info()

# Vérifier les statistiques descriptives
df1.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50253 entries, 0 to 50252
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   NBRE         50213 non-null  float64       
 1   MCR          50213 non-null  float64       
 2   MDB          50213 non-null  float64       
 3   VALEUR       50253 non-null  int64         
 4   DATECPS      50253 non-null  datetime64[ns]
 5   NUM          50253 non-null  int64         
 6   MNT          49521 non-null  float64       
 7   ID           50253 non-null  int64         
 8   DNA          31752 non-null  object        
 9   PROVISIONNE  21391 non-null  float64       
 10  PROFESSION   41528 non-null  object        
 11  LIBCCL       50236 non-null  object        
 12  SORT         50253 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(3), object(4)
memory usage: 5.0+ MB


,NBRE,MCR,MDB,VALEUR,DATECPS,NUM,MNT,ID,PROVISIONNE
count,50213.000000,5.021300e+04,5.021300e+04,50253.0,50253,5.025300e+04,4.952100e+04,5.025300e+04,21391.0
mean,312.347639,4.417925e+06,5.428699e+06,30.0,2024-12-15 07:52:26.104710144,6.497059e+06,3.437451e+04,3.877509e+06,1.0
min,13.000000,0.000000e+00,1.149248e+03,30.0,2024-10-01 00:00:00,1.900000e+01,6.000000e-02,3.690000e+02,1.0
25%,54.000000,4.646420e+04,4.560294e+04,30.0,2024-11-04 00:00:00,6.351813e+06,4.656000e+02,1.097830e+06,1.0
50%,69.000000,9.110710e+04,9.124224e+04,30.0,2024-12-09 00:00:00,6.733913e+06,2.754200e+03,4.590973e+06,1.0
75%,141.000000,2.786869e+05,2.827489e+05,30.0,2025-01-15 00:00:00,7.000649e+06,4.986700e+03,6.192783e+06,1.0
max,113018.000000,2.541437e+09,2.537465e+09,30.0,2025-04-25 00:00:00,7.995979e+06,4.600000e+07,6.988269e+06,1.0
std,2121.423428,4.998041e+07,5.267235e+07,0.0,NaN,9.812790e+05,7.290768e+05,2.477617e+06,0.0


In [4]:
# Vérifier les premières lignes
df2.head()

# Vérifier les informations sur le dataset
df2.info()

# Vérifier les statistiques descriptives
df2.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8927 entries, 0 to 8926
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   NBRE        8864 non-null   float64       
 1   MCR         8864 non-null   float64       
 2   MDB         8864 non-null   float64       
 3   VALEUR      8927 non-null   int64         
 4   DATECPS     8927 non-null   datetime64[ns]
 5   NUM         0 non-null      float64       
 6   MNT         8925 non-null   float64       
 7   ID          8901 non-null   float64       
 8   DNA         8540 non-null   datetime64[ns]
 9   PROFESSION  8195 non-null   object        
 10  LIBCCL      8858 non-null   object        
 11  SORT        8927 non-null   object        
dtypes: datetime64[ns](2), float64(6), int64(1), object(3)
memory usage: 837.0+ KB


,NBRE,MCR,MDB,VALEUR,DATECPS,NUM,MNT,ID,DNA
count,8864.000000,8.864000e+03,8.864000e+03,8927.000000,8927,0.0,8925.000000,8.901000e+03,8540
mean,43.807310,4.827420e+04,4.738282e+04,40.996863,2025-01-09 08:47:38.384675840,NaN,1569.842667,8.754051e+10,1979-04-19 18:56:21.217798656
min,1.000000,0.000000e+00,0.000000e+00,40.000000,2024-10-01 00:00:00,NaN,24.500000,3.690000e+02,1900-06-20 00:00:00
25%,19.000000,5.423027e+03,5.247775e+03,41.000000,2024-12-02 00:00:00,NaN,90.700000,1.858184e+06,1970-11-18 00:00:00
50%,27.000000,9.717702e+03,8.963404e+03,41.000000,2025-01-16 00:00:00,NaN,143.430000,3.922730e+06,1980-11-02 00:00:00
75%,47.000000,1.282768e+04,1.309075e+04,41.000000,2025-02-28 00:00:00,NaN,500.000000,5.833789e+06,1987-07-06 12:00:00
max,1057.000000,4.486243e+06,4.458645e+06,41.000000,2025-03-28 00:00:00,NaN,75000.000000,1.881328e+13,2004-11-16 00:00:00
std,77.377669,3.178323e+05,3.150377e+05,0.055920,NaN,NaN,4883.869200,1.274948e+12,NaN


In [6]:
# Remplacer NaN dans la colonne PROVISIONNE par 0
df1['PROVISIONNE'] = df1['PROVISIONNE'].fillna(0).astype(int)

# Remplacer NaN dans la colonne PROFESSION par AUTRE PROFESSION
df1['PROFESSION'] = df1['PROFESSION'].fillna('AUTRE PROFESSION')

df1.isna().sum()

NBRE              40
MCR               40
MDB               40
VALEUR             0
DATECPS            0
NUM                0
MNT              732
ID                 0
DNA            18501
PROVISIONNE        0
PROFESSION         0
LIBCCL            17
SORT               0
dtype: int64

In [ ]:
# Vérification du type des colonnes
df1['MCR'] = df1['MCR'].astype(str).str.replace(' ', '').str.replace(',', '.')
df1['MDB'] = df1['MDB'].astype(str).str.replace(' ', '').str.replace(',', '.')
df1['MNT'] = df1['MNT'].astype(str).str.replace(',', '.')

# Conversion en float
df1['MCR'] = pd.to_numeric(df1['MCR'], errors='coerce')
df1['MDB'] = pd.to_numeric(df1['MDB'], errors='coerce')
df1['MNT'] = pd.to_numeric(df1['MNT'], errors='coerce')

# Calculer l'âge à partir de la date de naissance (DNA)
df1['DNA'] = pd.to_datetime(df1['DNA'], errors='coerce')
df1['AGE'] = (pd.Timestamp.today() - df1['DNA']).dt.days // 365

In [ ]:
# Remplacer NaN dans la colonne LIBCCL par Personne physique (la meme personne)
df1['LIBCCL'] = df1['LIBCCL'].fillna('Personne physique')
df1.isna().sum()

In [ ]:
# Encodage des variables catégorielles (PROFESSION, LIBCCL)
le_profession = LabelEncoder()  # Création de l'encodeur
df1['le_PROFESSION'] = le_profession.fit_transform(df1['PROFESSION'].fillna(''))

le_libccl = LabelEncoder()  # Création de l'encodeur pour LIBCCL
df1['le_LIBCCL'] = le_libccl.fit_transform(df1['LIBCCL'].fillna(''))

# Encoder la variable cible 'SORT'
df1['SORT'] = df1['SORT'].map({'P': 0, 'CNP': 1, 'AUTRE REJET': 1, 'PREAVIS': 1})

In [ ]:
# Select only numeric columns
numeric_cols = df1.select_dtypes(include=['number']).columns

# Compute skewness only on numeric columns
skew_values = df1[numeric_cols].skew()

print("Skewness of numeric columns:")
print(skew_values)


In [ ]:
# Manual list from your skewness output
median_cols = ['NBRE', 'MCR', 'MDB', 'MNT', 'SORT', 'le_PROFESSION', 'le_LIBCCL', 'NUM']
mean_cols = ['VALEUR', 'ID', 'PROVISIONNE', 'AGE']

# Impute using median for skewed columns
for col in median_cols:
    df1[col] = df1[col].fillna(df1[col].median())
    print(f"{col}: Filled NaN with median (high skew)")

# Impute using mean for relatively normal columns
for col in mean_cols:
    df1[col] = df1[col].fillna(df1[col].mean())
    print(f"{col}: Filled NaN with mean (low skew)")


In [ ]:
# Sélectionner les variables explicatives (X) et la cible (y)
X = df1[['NBRE', 'MCR', 'MDB', 'VALEUR', 'MNT', 'AGE', 'PROVISIONNE', 'le_PROFESSION', 'le_LIBCCL']]
y = df1['SORT']

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier  # Replace with your model

# Initialize Stratified K-Fold
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Example model
model = RandomForestClassifier(random_state=42)

# Loop over the folds
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n--- Fold {fold} ---")
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import numpy as np

# Setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Models to evaluate
models = {
    "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000)
}

# Results storage
results = {name: {"accuracy": [], "f1": []} for name in models}

# Cross-validation
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n=== Fold {fold} ===")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')  # use 'binary' if binary classification

        results[name]["accuracy"].append(acc)
        results[name]["f1"].append(f1)

        print(f"\n- {name}")
        print("Matrice de confusion :\n", confusion_matrix(y_test, y_pred))
        print("Rapport de classification :\n", classification_report(y_test, y_pred))

# Average results
print("\n=== Résultats Moyens sur les 5 Folds ===")
for name, scores in results.items():
    avg_acc = np.mean(scores["accuracy"])
    avg_f1 = np.mean(scores["f1"])
    print(f"{name}: Accuracy moyenne = {avg_acc:.4f} | F1-score moyen = {avg_f1:.4f}")


In [ ]:
# Fonction pour visualiser la matrice de confusion
def plot_conf_matrix(cm, model_name):
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

# Models to evaluate
models = {
    "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight="balanced")#,
    #"XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss")  # Optional
}

# Results storage
results = {name: {"accuracy": [], "f1": [], "roc_auc": [], "avg_precision": []} for name in models}

# Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n=== Fold {fold} ===")
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Apply SMOTE
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

    for name, model in models.items():
        model.fit(X_train_res, y_train_res)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]

        # Metrics
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average="macro")
        roc_auc = roc_auc_score(y_test, y_proba)
        avg_prec = average_precision_score(y_test, y_proba)

        # Store
        results[name]["accuracy"].append(acc)
        results[name]["f1"].append(f1)
        results[name]["roc_auc"].append(roc_auc)
        results[name]["avg_precision"].append(avg_prec)

        # Output
        print(f"\n- {name}")
        print("Matrice de confusion :\n", confusion_matrix(y_test, y_pred))
        print("Rapport de classification :\n", classification_report(y_test, y_pred))

        # Plot
        plot_conf_matrix(confusion_matrix(y_test, y_pred), f"{name} - Fold {fold}")

# === Résumé des performances ===
print("\n=== Résultats Moyens sur les 5 Folds ===")
for name, scores in results.items():
    print(f"\n{name}:")
    print(f" - Accuracy moyenne       = {np.mean(scores['accuracy']):.4f}")
    print(f" - F1-score moyen         = {np.mean(scores['f1']):.4f}")
    print(f" - ROC AUC moyen          = {np.mean(scores['roc_auc']):.4f}")
    print(f" - Average Precision mean = {np.mean(scores['avg_precision']):.4f}")




## 🧑‍💻 Model Performance Results (5-Fold Cross-Validation)

### KNN (k=5)
- **Accuracy**: 97.02%
- **F1-score**: 0.6630
- **ROC AUC**: 0.8410
- **Average Precision**: 0.3277

### Random Forest
- **Accuracy**: 98.86%
- **F1-score**: 0.7801
- **ROC AUC**: 0.8723
- **Average Precision**: 0.5115

### Logistic Regression
- **Accuracy**: 70.29%
- **F1-score**: 0.4368
- **ROC AUC**: 0.7160
- **Average Precision**: 0.0324

---

## 🚀 Conclusion:
- **Random Forest** is the best model, achieving:
  - **Highest ROC AUC** (0.8723), showing excellent class distinction.
  - **Highest F1-score** (0.7801), balancing precision and recall.
  - **Better Average Precision** (0.5115), indicating better detection of the minority class compared to KNN and Logistic Regression.

- **KNN (k=5)** offers decent performance but falls behind Random Forest in handling the class imbalance and achieving a higher F1-score and Average Precision.

- **Logistic Regression** performs poorly due to the class imbalance, with low accuracy, F1-score, and Average Precision.


In [ ]:
# client_table = cheque_overview[["ID","NBRE", "MCR", "MDB", "AGE", "PROFESSION", "LIBCCL"]]
# client_table.drop_duplicates(subset="ID", inplace=True)
# client_table.shape

# client_table.to_excel("data/client_table.xlsx", index=False)
